**This program will parse your data of midi files into data readable by the ai**

Import

In [1]:
!pip install py_midicsv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Upload your zip file of midi files (all in root of zip) and rename to music.zip also create a folder under the same name in the content folder**

In [8]:
!unzip /content/music.zip -d /content/music/

Archive:  /content/music.zip
  inflating: /content/music/epic country.mid  


In [71]:
#@title # **Convert the dataset and make sure to change midi dir if you changed it above** *Also* you may get errors but its probably ok
import py_midicsv as pm
import os


midi_dir = "/content/music/" #@param {type:"string"}
midi_files = [f for f in os.listdir(midi_dir) if f.endswith('.mid')]
csv_string = []

def get_note_event(start, note, instrument, length, velocity):
  outValue = str(note)

  if(instrument<=127):
    outValue += "%"

  outValue += str(start)

  if(velocity > 108):
    outValue += "$"
  elif velocity > 74:
    outValue += "#"
  elif velocity > 40:
    outValue += "@"
  else:
    outValue += "!"

  outValue += str(length)

  return outValue

def parse_split_csv_item(item):
  try:
    item[2] = item[2].replace(' ', '')
    item[0] = int(item[0])
    item[1] = int(item[1])
    item[3] = int(item[3])
    item[4] = int(item[4])
    item[5] = int(item[5])
  except:
    pass;
  return item

# Load the MIDI file and parse it into CSV format
for midi_file in midi_files:
  try:
    lastNoteStart = 0
    csv_unparsed = pm.midi_to_csv(midi_dir + midi_file)
    csv_parsed = []
    notesPlaying = []
    for csv_item in csv_unparsed:
      split_csv_item = csv_item.split(',')
      split_csv_item = parse_split_csv_item(split_csv_item);
      match(split_csv_item[2]):
        case "Note_on_c":
          notesPlaying.append(split_csv_item)
        case "Note_off_c":
          # Find note that turning off
          for i in range(len(notesPlaying)):
            if(notesPlaying[i][4] == split_csv_item[4]):
              csv_string += get_note_event(notesPlaying[i][1] - lastNoteStart, split_csv_item[4], 0, split_csv_item[1] - notesPlaying[i][1], notesPlaying[i][5]) + '\n'
              lastNoteStart = notesPlaying[i][1]
              notesPlaying.pop(i)
  except Exception as e:
    print(e)

with open("convertedData.txt", "w") as f:
    f.writelines(csv_string)

list index out of range


#Ignore this

In [ ]:
import py_midicsv as pm

csv_string = "/content/temp3.csv"

# Parse the CSV output of the previous command back into a MIDI file
midi_object = pm.csv_to_midi(csv_string)

# Save the parsed MIDI file to disk
with open("aitest12.mid", "wb") as output_file:
    midi_writer = pm.FileWriter(output_file)
    midi_writer.write(midi_object)